# Components with hierarchy

You can define some components (waveguides, bends, couplers) as a stand alone components, with basic input parameters (width, length, radius ...)

Then you can re-use those components in a more complex hiearchical components.

`gdsfactory` does this by passing the higher level component with the lower level functions to build the components.

You can customize any of the functions thanks to `functools.partial`

In [ ]:
import gdsfactory as gf

In [ ]:
@gf.cell
def mzi_sample(
    splitter=gf.c.mmi1x2,
    combiner=gf.c.mmi1x2,
    bend=gf.c.bend_euler,
    straight=gf.c.straight,
    delta_length: float = 100.0,
):
    arm_top_sequence = "BR-RB"
    arm_top_symbol_to_component = {
        "B": (bend(), "o1", "o2"),
        "R": (bend(), "o2", "o1"),
        "-": (straight(), "o1", "o2"),
        "_": (straight(length=delta_length / 2), "o1", "o2"),
    }
    arm_top_sequence = "B_R-R_B"
    arm_bot_sequence = "RB-BR"
    arm_top = gf.c.component_sequence(
        sequence=arm_top_sequence, symbol_to_component=arm_top_symbol_to_component
    )
    arm_bot = gf.c.component_sequence(
        sequence=arm_bot_sequence, symbol_to_component=arm_top_symbol_to_component
    )

    c = gf.Component()
    top = c << arm_top
    bot = c << arm_bot
    splitter = c << splitter()
    combiner = c << combiner()

    top.connect("o1", splitter.ports["o2"])
    bot.connect("o1", splitter.ports["o3"])
    combiner.connect("o2", bot.ports["o2"])
    c.add_port("o1", port=splitter.ports["o1"])
    c.add_port("o2", port=combiner.ports["o1"])
    return c


c = mzi_sample()
c

Lets **customize** the functions that we pass.
For example, we want to increase the radius of the bend from the default 10um to 50um.

In [ ]:
gf.c.bend_circular()

## functools.partial

Partial lets you define different default parameters for a function, so you can modify the settings for the child cells.

In [ ]:
from functools import partial

In [ ]:
bend50 = partial(gf.c.bend_circular, radius=50)
bend50()

In [ ]:
type(bend50)

In [ ]:
bend50.func.__name__

In [ ]:
bend50.keywords

In [ ]:
c = mzi_sample(bend=partial(gf.c.bend_circular, radius=50))
c

In [ ]:
bend50(radius=10)

In [ ]:
c = mzi_sample()
c

## PDK custom fab

You can define a new PDK by creating function that customize partial parameters of the generic functions.

Lets say that this PDK uses layer (41, 0) for the pads (instead of the layer used in the generic pad function).

You can also access `functools.partial` from `gf.partial`

In [ ]:
import gdsfactory as gf

pad = gf.partial(gf.c.pad, layer=(41, 0))

In [ ]:
pad()

## Composing functions

You can combine more complex functions out of smaller functions.

Lets say that we want to add tapers and grating couplers to a wide waveguide.

In [ ]:
c1 = gf.c.straight()
c1

In [ ]:
straight_wide = gf.partial(gf.components.straight, width=3)
c3 = straight_wide()
c3

In [ ]:
c1 = gf.c.straight(width=3)
c1

In [ ]:
c2 = gf.add_tapers(c1)
c2

In [ ]:
c3 = gf.routing.add_fiber_array(c2, with_loopback=False)
c3

Lets do it with a **single** step thanks to `toolz.pipe`

In [ ]:
import toolz

add_fiber_array = gf.partial(gf.routing.add_fiber_array, with_loopback=False)
add_tapers = gf.add_tapers

# pipe is more readable than the equivalent add_fiber_array(add_tapers(c1))
c3 = toolz.pipe(c1, add_tapers, add_fiber_array)
c3

we can even combine `add_tapers` and `add_fiber_array` thanks to `toolz.compose` or `toolz.compose`

In [ ]:
add_tapers_fiber_array = toolz.compose_left(add_tapers, add_fiber_array)
add_tapers_fiber_array(c1)

In [ ]:
add_tapers_fiber_array = toolz.compose(add_fiber_array, add_tapers)
add_tapers_fiber_array(c1)

In [ ]:
toolz.pipe?

In [ ]:
toolz.compose?